# 神经网络基础与fasttext

## 神经网络

由非线性激活函数的神经元组成，其通过大量神经元之间的连接，成为高度非线性的模型。神经元之间的连接权重就是需要学习的参数，可以通过梯度下降方法来进行学习。

## 激活函数

性质：连续并可导(允许少数点上不可导)的非线性函数。可导的激活函数可以直接利用数值优化的方法来学习网络参数。 激活函数及其导函数要尽可能的简单，有利于提高网络计算效率。激活函数的导函数的值域要在一个合适的区间内，不能太大也不能太小，否则会影响训练的效率和稳定性。

### 1 Sigmoid 型激活函数
#### 1 logistic函数
![Sigmoid](1.png)
特点：
（1）其输出直接可以看作是概率分布，使得神经网络可以更好地和统计学习模型进行结合。
（2）其可以看作是一个软性门(Soft Gate)，用来控制其它神经元输出信息的数量 。

#### 2 Tanh函数
![Tanh](2.png)

两个函数的关系
![R](3.png)
Tanh函数输出零中心化的(Zero-Centered)，Logistic函数的输出恒大于 0。非零中心化的输出会使得其后一层的神经元的输入发生偏置偏移(Bias Shift)，并进一步使得梯度下降的收敛速度变慢。

### 2 Relu函数
![Relu](4.png)

优点：采用ReLU的神经元只需要进行加、乘和比较的操作，计算上更加高效。ReLU函数被认为有生物上的解释性，比如单侧抑制、宽兴奋边界Sigmoid型激活函数会导致一个非稀疏的神经网络，而 ReLU却具有很好的稀疏性，大约50%的神经元会处于激活状态。相比于 Sigmoid 型函数的两端饱和，ReLU函数为左饱和函数，且在x > 0时导数为1，在一定程度上缓解了神经网络的梯度消失问题，加速梯度下降的收敛速度。

缺点： ReLU函数的输出是非零中心化的，给后一层的神经网络引入偏置偏移，会影响梯度下降的效率。如果参数在一次不恰当的更新后，第一个隐藏层中的某个ReLU 神经元在所有的训练数据上都不能被激活，那么这个神经元自身参数的梯度永远都会是0，在以后的训练过程中永远不能被激活。

# 2 前馈神经网络

前馈神经网络也称为多层感知器(Multi-Layer Perceptron，MLP)。但多层感知器的叫法并不是十分合理，前馈神经网络是由多层的 Logistic 回归模型(连续的非线性函数)组成，而不是由多层的感知器(不连续的非线性函数)组成 。
![bp](5.png)

通用近似定理：对于具有线性输出层和至少一个使用“挤压”性质的激活函数的隐藏层组成的前馈神经网络，只要其隐藏层神经元的数量足够，它可以以任意的精度来近似任何从一个定义在实数空间 Rd 中的有界闭集函数。
要取得好的分类效果，需要样本的原始特征向量 x 转换到更有效的特征向量 φ(x)，这个过程叫做特征抽取。

两类分类问题y=0/1，并采用Logistic回归，那么Logistic回归分类器可以看成神经网络的最后一层。网络的最后一层只用一个神经元，并且其激活函数为Logistic函数。网络的输出可以直接可以作为类别y = 1的后验概率。 p(y = 1|x) = a(L),其中a(L) ∈ R为第L层神经元的活性值。

对于多类分类问题y∈{1...C}，如果使用softmax 回归分类器，相当于网络最后一层设置 C个神经元，其激活函数为softmax函数。网络的输出可以作为每个类的后验概率。 yˆ = softmax(z(L)), 其中z(L) ∈ R为第L层神经元的净输入;yˆ ∈ RC 为第L层神经元的活性值，分别是不同类别标签的预测后验概率。


# fasttext

fasttext 是一个用于高效学习单词表示和句子分类的库。
与基于神经网络的分类算法相比有两大优点：
（1）fasttext在保持高精度的情况下加快了训练速度和测试速度。
（2）fasttext不需要预训练好的词向量，fasttext会自己训练词向量。

## fasttext原理

fasttext包含三部分，模型架构，层次SoftMax和N-gram子词特征。
fasttext模型架构和word2vec中的CBOW很相似， 不同之处是fasttext预测标签而CBOW预测的是中间词，即模型架构类似但是模型的任务不同。
fasttext模型架构如下图，其中x1,x2,…,xN−1,xN表示一个文本中的n-gram向量，每个特征是词向量的平均值。
![ft](6.png)
对于有大量类别的数据集，fasttext使用了一个分层分类器（而非扁平式架构）不同的类别被整合进树形结构中。在某些文本分类任务中类别很多，计算线性分类器的复杂度高。为了改善运行时间，fasttext 模型使用了层次 Softmax 技巧。层次 Softmax 技巧建立在哈弗曼编码的基础上，对标签进行编码，能够极大地缩小模型预测目标的数量。
fasttext 也利用了类别（class）不均衡这个事实（一些类别出现次数比其他的更多），通过使用 Huffman 算法建立用于表征类别的树形结构。因此，频繁出现类别的树形结构的深度要比不频繁出现类别的树形结构的深度要小，这也使得进一步的计算效率更高。

fasttext 使用n-gram有如下优点:
(1)为罕见的单词生成更好的单词向量。
(2)在词汇单词中，即使单词没有出现在训练语料库中，仍然可以从字符级n-gram中构造单词的词向量。
(3)n-gram可以让模型学习到局部单词顺序的部分信息。


In [4]:
import fasttext as ft
classifier = ft.supervised("fasttext_train.txt","fasttext.model",label_prefix = "__label__")
# 加载模型
classifier = ft.load_model("fasttext.model.bin",label_prefix = "__label__")

# 测试模型 其中 fasttext_test.txt 就是测试数据，格式和 fasttext_train.txt 一样
result = classifier.test("fasttext_test.txt")
print("准确率:",result.precision)
print("回归率:",result.recall)

# 使用模型,以测试集中第一个文档为例
line = ["Youth is not a time of life; it is a state of mind. It is not a matter of rosy cheeks, red lips and supple knees. It is a matter of the will, a quality of the imagination, vigor of the emotions; it is the freshness of the deep spring of life.",
        "Youth means a temperamental predominance of courage over timidity, of the appetite for adventure over the love of ease. This often exits in a man of 60, more than a boy of 20. Nobody grows merely by the number of years; we grow old by deserting our ideas. Years may wrinkle the skin, but to give up enthusiasm wrinkles the soul. Worry, fear, self-distrust bows the heart and turns the spirit back to dust.",
        "Whether 60 or 16, there is in every human being’s heart the lure of wonders, the unfailing childlike appetite of what’s next and the joy of the game of living. In the center of your heart and my heart there is a wireless station; so long as it receives messages of beauty, hope, cheer, courage and power from men and from infinite, so long as you are young.",
        "When the aerials are down, and your spirit is covered with the snows of cynicism and the ice of pessimism, then you’ve grown old, even at 20, but as long as your aerials are up, to catch waves of optimism, there’s hope you may die young at 80."]


result = classifier.predict([line])
print(result)

ModuleNotFoundError: No module named 'fasttext'